<div align="center" dir="auto">
<p dir="auto">

<a href="https://colab.research.google.com/github/write-with-neurl/modelbit-articles/blob/main/modelbit-04/code/Deploy_Llama_2_7b_Text_Summarization_LangChain_Modelbit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

</p>

# ⚡Deploy Llama 2-7B 🦙 as a REST Endpoint with Langchain 🦜 and Modelbit

## 🧑‍💻 Installations and Set Up

This walkthrough will guide you through deploying a Llama 2-7B 🦙 directly from this notebook using Modelbit.





### 📥 Install Pre-requisite Libraries

To set up your environment, install the following packages:
- LangChain - `langchain==0.0.335`
- Modelbit - `modelbit==0.30.13`
- Huggingface_hub - `huggingface-hub==0.19.1`
- LLaMA_cpp_python - `llama_cpp_python==0.2.17`

As of this writing, those are the specific versions the following `pip` command installs:


In [1]:
!pip install --upgrade langchain huggingface_hub modelbit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.8/113.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of types-requests to determine which version is compatible with ot

### 🏗️ Build  Llama 2-7b 🦙 with GPU Support

You will load the embedding model directly onto your GPU device. Set an environment variable `CMAKE_ARGS` with the value `-DLLAMA_CUBLAS=on` to indicate that the `llama_cpp_python` package should be built with [cuBLAS support](https://developer.nvidia.com/cublas).

> The `llama_cpp_python` package provides Python bindings for the llama.cpp library that bridges the gap between the C++ codebase of `llama.cpp` and Python to access and use the functionalities of llama.cpp directly from Python scripts.

cuBLAS is a GPU-accelerated library provided by NVIDIA as part of their CUDA toolkit, which offers optimized implementations for standard basic linear algebra subprograms.

In [2]:
CMAKE_ARGS="-DLLAMA_CUBLAS=on"
!pip install llama_cpp_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama_cpp_python: filename=llama_cpp_python-0.2.17-cp310-cp310-manylinux_2_35_x86_64.whl size=1894426 sha256=d6c85dbbacfc643a272d3fc7df7d110dc868f77b8508fcb5862af89e14245166
  Stored in directory: /root/.cache/pip/wheels/a2/4e/34/8fc83f26e268f1957f8a0ebfa763cfb80af31d600b569d7680
Successfully built llama_cpp_python


### 📥🦙 Download the Llama-2-7B-GGUF model using the Hugging Face CLI command

Use the HF CLI to download the Llama-2-7B-GGUF model file from a repository in the Hugging Face model Hub and save it to this notebook environment.

In [3]:
!huggingface-cli download TheBloke/Llama-2-7B-GGUF llama-2-7b.Q4_0.gguf --local-dir . --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q4_0.gguf to /root/.cache/huggingface/hub/tmpswmww8dd
llama-2-7b.Q4_0.gguf: 100% 3.83G/3.83G [01:32<00:00, 41.4MB/s]
./llama-2-7b.Q4_0.gguf


## 🦜🔗 Set Up the Prompt Template with LangChain



After downloading the model file, you need to set up a [prompt template](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/). The prompt template defines the input variables and the response format for the LlamaCpp model.

In this case, the input variables are `text` and `num_of_words`, which represent the Twitter thread and the desired length of the summary, respectively.

The response format is a `str` that includes the original Twitter thread and the generated summary.

In [4]:
from langchain.prompts import PromptTemplate


prompt = PromptTemplate(
    input_variables=["text", "num_of_words"],
    template="""\n\n### Instruction:\nGiven a twitter thread: '{text}', You are tasked with summarizing the twitter thread in {num_of_words} words and ensure the summary doesn't lose the main context of the twitter thread. \n\n### Response:\n""",
)

## 🦙⛓️ initialize Llama Model Weights and LLMChain




After crafting your prompt, initialize the LlamaCpp model with the model file path and context size(`n_ctx`).

Use the prompt template to create an LLMChain class with the initialized LlamaCpp model and prompt template.

Finally, the `load_llm` function is defined to load the LlamaCpp model embeddings/weights and cache it for future use. Then we deifne a function `summarize_thread` that takes a text string and a number of words as input, and generates a summary of the text.



In [5]:
# Importing necessary classes and modules
from langchain.chains import LLMChain  # Used for creating language model chains
from langchain.callbacks.stdout import StdOutCallbackHandler  # Handles output to standard output
import os  # Used for operating system dependent functionality
from langchain.llms.llamacpp import LlamaCpp  # Interface for LlamaCpp model
from functools import cache  # Used for caching function results

# Setting up the file path for the model
folder = ''  # Directory where the model file is located (empty if in current directory)
file_name = 'llama-2-7b.Q4_0.gguf'  # Name of the model file
file_path = os.path.join(folder, file_name)  # Full path to the model file

@cache
def load_llm():
    """ Loads the LlamaCpp model with specified model path and context size.
        Uses caching (`@cache`) to optimize performance by storing the result for subsequent calls.
    """
    llm = LlamaCpp(model_path=file_path, n_ctx=8191) # Load the LlamaCpp model
    return llm



### 📜 Define Inference Function

This function takes a text string and an optional `num_of_words argument` (defaulting to 200).

`llm = load_llm()` - calls the `load_llm` function to get the loaded LlamaCpp model.

`chain = LLMChain(llm=llm, prompt=prompt)` - Instantiates an LLMChain object with the LlamaCpp model and a prompt.

`return chain.run(...)` - executes the chain with the specified parameters and returns the result. The run method is passed the text and number of words, along with a callback handler `(StdOutCallbackHandler())` which  handles the output.

In [6]:
def summarize_thread(text: str, num_of_words: int = 200):
    """ Summarizes the given text using the LlamaCpp model.

    Args:
        text (str): Text to be summarized.
        num_of_words (int, optional): Number of words for the summary. Defaults to 200.

    Returns:
        The summary of the text.
    """
    llm = load_llm()  # Load the LlamaCpp model
    # Initialize LLMChain with the LlamaCpp model and the provided text as prompt

    chain = LLMChain(llm=llm, prompt=prompt)
    # Run the chain with the text and word limit, and return the result
    return chain.run({"text": text, "num_of_words":num_of_words}, callbacks=[StdOutCallbackHandler()])

## 🧪 Test the Inference Function with a Sample Twitter Thread 🧵

In [7]:
text="""In today's fast-paced world, it's crucial to stay connected and informed. Communication has evolved over the years, with technology playing a pivotal role. We're now able to share ideas, news, and personal stories with a global audience instantly.

The rise of social media has been a game-changer."""

# call inference function
summarize_thread(text=text, num_of_words=200)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 




> Entering new LLMChain chain...
Prompt after formatting:


### Instruction:
Given a twitter thread: 'In today's fast-paced world, it's crucial to stay connected and informed. Communication has evolved over the years, with technology playing a pivotal role. We're now able to share ideas, news, and personal stories with a global audience instantly.

The rise of social media has been a game-changer.', You are tasked with summarizing the twitter thread in 200 words and ensure the summary doesn't lose the main context of the twitter thread. 

### Response:


> Finished chain.


"The rise of social media is truly remarkable, transforming how we connect and share information today. Twitter, a microblogging platform, has played a significant role in this revolution. People can now share short messages, known as tweets, with a massive audience, instantly communicating ideas, opinions, news, and personal stories worldwide.\n\nThe evolution of communication is unparalleled in the modern era. Twitter has become an indispensable tool for connecting people from all walks of life. As more and more individuals use this platform to express their thoughts, it's increasingly important to stay informed about what's happening around us so that we can contribute positively to society.\n\nTwitter is now a global phenomenon that continues to shape our world significantly. It provides us with invaluable information regarding current events and trends. We remain connected through this powerful medium of communication, facilitating dialogue and understanding among people from vari

## 🚀 Deploying Llama 2-7B 🦙 as a REST API Endpoint with Modelbit



Modelbit is a lightweight platform designed to make deploying any ML model to a production endpoint fast and simple. With the ability to [deploy models from anywhere](https://www.modelbit.com/product/deploy-from-anywhere), it makes deploying your custom ML model as simple as passing an inference function to “modelbit.deploy()”.

Here are the basics you need to know about Modelbit:
- **Deployment from any Python environment:** Models can be deployed directly from Jupyter Notebooks, Hex, Deepnote, and VS Code.
- **Dependency detection:** Automatically detects which dependencies, libraries, and data your model needs and includes them in your model’s production Docker container.
- **REST API Endpoints:** Your models will be callable as [RESTful API endpoints](https://doc.modelbit.com/endpoints/).
- **Git-based version control:** Track and manage model iterations with Git repositories.
- **CI/CD integration:** Seamlessly integrate model updates and deployment into continuous integration and continuous delivery (CI/CD [link text](https://doc.modelbit.com/git/#connect-your-own-github-gitlab-or-azure-devops-repo)) pipelines like [GitHub Actions](https://github.com/features/actions) and [GitLab CI/CD](https://about.gitlab.com/solutions/continuous-integration/).

## 🔒 Log into Modelbit

Modelbit integrates with version control and exepriment iteration tools like gitlab, github, Weights & Biases and neptune.ai to move models from development to production quickly.

 To get started;

Create an account with [Modelbit]('modelbit.com') if you haven't already.**

In [7]:
import modelbit

# Log into the 'modelbit' service using the development ("dev") branch
# Ensure you create a "dev" branch in Modelbit or use the "main" branch for your deployment
mb = modelbit.login(branch="dev")

## ⚡ Time to `modelbit.deploy()`

Deploy the `summarize_thread` function to Modelbit with the [`mb.deploy()`](https://doc.modelbit.com/api-reference/deploy/) API.

This API takes the inference function as an argument and deploys it to Modelbit, making it available through a REST API.

Once deployed, you can use the REST API to call the function and generate summaries of Twitter threads.

In [8]:
mb.deploy(summarize_thread, extra_files=["llama-2-7b.Q4_0.gguf"], python_packages=["langchain==0.0.335","llama_cpp_python==0.2.17"],
          require_gpu=True)

Uploading 'prompt': 100%|██████████| 400/400 [00:00<00:00, 727B/s]


## 🧑‍🍳 Test the Llama 2-7B 🦙 REST API Endpoint



To test your REST Endpoint, you can use the requests package to send single or batch requests to the API.

You can use the `requests.post()` method to send a POST request to the API and use the json module to format the response. Here's an example of how to test your REST Endpoint using Python

In [9]:
twitter_thread="""In today's fast-paced world, it's crucial to stay connected and informed. Communication has evolved over the years, with technology playing a pivotal role. We're now able to share ideas, news, and personal stories with a global audience instantly.

The rise of social media has been a game-changer."""

Test your endpoint from the command line using:

> ⚠️ Replace the `ENTER_WORKSPACE_NAME` placeholder with your workspace name.

In [19]:
"""Testing the endpoint"""
import requests
import json

# Replace “ENTER_WORKSPACE_NAME” with your Modelbit username
# Also change the version branch from `v1` if you have a different deployment version

url = "https://ENTER_WORKSPACE_NAME.app.modelbit.com/v1/summarize_thread/dev/latest"


headers = {
    'Content-Type': 'application/json'
}
data = {
    "data": [twitter_thread, 200] # text and num_of_words argument for inf function
}


# POST your request to the endpoint
response = requests.post(url, headers=headers, json=data)
response_json = response.json()

print(json.dumps(response_json, indent=4))

{
    "data": "Twitter is one of the most popular platforms for users to stay connected and informed about current events, news and personal stories around the world. It allows its users to share ideas instantly with a global audience in real-time through its messaging system. With technology playing a vital role in the platform's evolution over time, Twitter has become one of the most important communication tools available today.\n\nThe rise of social media platforms such as Facebook and Instagram have significantly impacted how people communicate with each other online and offline alike due to their ability to deliver content quickly across multiple devices simultaneously while providing users with immediate access anywhere they may be located at any given moment in time \u2013 making communication more accessible than ever before!\n\n### Instruction:\nGiven a twitter thread: 'Twitter has become an important communication tool for millions of people around the world.', You are taske

** Use `cURL` **

> ⚠️ Replace the `ENTER_WORKSPACE_NAME` placeholder with your workspace name.

In [ ]:
!curl -s -XPOST "https://ENTER_WORSKPACE_NAME.app.modelbit.com/v1/resnet_inference/dev/latest" -d '{"data": [twitter_thread, 200]}' | json_pp

# 📚 Modelbit Machine Learning Blog

Enjoyed this walkthrough? Check out our blog for more machine learning tutorials.

Recommendation:

- [Deploying a BERT Model to a REST API Endpoint for Text Classification](https://www.modelbit.com/blog/deploying-a-bert-model-to-a-rest-api-endpoint-for-text-classification).